## 테스트 작업

In [8]:
import os
import numpy as np
import pandas as pd

import cx_Oracle

In [9]:
# CSV 데이터 컬럼명

DATA_FILE_COLUMNS = ['region', 'region_sub', 'apt_name', 'apt_size',
                     'apt_floor', 'contract_date', 'contract_price',
                     'completion_year', 'street_num', 'street_name']

# region = 도, 광역시, 특별시
# region_sub = 시군구
# apt_name = 단지명
# apt_size = 전용면적
# apt_floor = 층
# contract_date = 계약년월일
# contract_price = 거래금액
# completion_year = 건축년도
# street_num = 번지
# street_name = 도로명

In [15]:
# 테이블 및 자동증가 시퀀스 생성 함수

def create_table_seq():
    # APT_DEAL_PRICE TABLE
    create_table_sql = """
        CREATE TABLE APT_DEAL_PRICE
        (
            APT_ID NUMBER PRIMARY KEY,
            REGION VARCHAR2(20 CHAR) NOT NULL,
            REGION_SUB VARCHAR2(40 CHAR) DEFAULT ('없음') NOT NULL,
            APT_NAME VARCHAR2(40 CHAR) NOT NULL,
            APT_SIZE NUMBER NOT NULL,
            APT_FLOOR NUMBER NOT NULL,
            CONTRACT_DATE DATE NOT NULL,
            CONTRACT_PRICE NUMBER NOT NULL,
            COMPLETION_YEAR NUMBER NOT NULL,
            STREET_NUM VARCHAR2(20 CHAR) DEFAULT('없음') NOT NULL,
            STREET_NAME VARCHAR2(40 CHAR) DEFAULT ('없음') NOT NULL
        )
        """

    # FOR APT_ID, AUTO INCREMENT ID
    create_seq_sql = "CREATE SEQUENCE SEQ_ID NOCACHE"

    conn = cx_Oracle.connect('hoseo', 'hoseo', 'localhost:1521/xe')
    cursor = conn.cursor()

    cursor.execute(create_table_sql)
    cursor.execute(create_seq_sql)

    conn.commit()
    cursor.close()
    conn.close()

In [11]:
# DB 확인하고 한 번만 실행!

# create_table_seq()

In [12]:
# CSV 파일 리스트 읽어오기, data-files 폴더

DATA_FILES_PATH = 'data-files'

data_files = os.listdir(DATA_FILES_PATH)
data_files = [ f'{DATA_FILES_PATH}/{r}' for r in data_files ]
data_files[0], len(data_files)

('data-files/201208.csv', 96)

In [13]:
# CSV 파일 로드 테스트

testD = pd.read_csv('data-files/201507.csv', encoding='utf-8')
testD['street_num'].fillna('없음', inplace=True)
testD['street_name'].fillna('없음', inplace=True)
testD.info()
# testD.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61775 entries, 0 to 61774
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   region           61775 non-null  object 
 1   region_sub       61775 non-null  object 
 2   apt_name         61775 non-null  object 
 3   apt_size         61775 non-null  float64
 4   apt_floor        61775 non-null  int64  
 5   contract_date    61775 non-null  object 
 6   contract_price   61775 non-null  int64  
 7   completion_year  61775 non-null  int64  
 8   street_num       61775 non-null  object 
 9   street_name      61775 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 4.7+ MB


In [14]:
# dataframe 튜플 리스트로 변환
testD_tp = [ tuple(r) for r in testD.values ]
testD_tp[0]

('강원도',
 '강릉시 견소동',
 '송정한신',
 84.94,
 10,
 '2015-07-01',
 138000000,
 1997,
 '202',
 '경강로2539번길 8')

In [17]:
# dataframe -> DB로 insert 하기 함수

def bulk_insert(tuple_data):
    bulk_insert_sql = """
        INSERT INTO APT_DEAL_PRICE
        VALUES (
            SEQ_ID.NEXTVAL, :REGION, :REGION_SUB, :APT_NAME,
            :APT_SIZE, :APT_FLOOR, :CONTRACT_DATE, :CONTRACT_PRICE,
            :COMPLETION_YEAR, :STREET_NUM, :STREET_NAME)
        """

    conn = cx_Oracle.connect('hoseo', 'hoseo', 'localhost:1521/xe')
    cursor = conn.cursor()

    try:
        cursor.prepare(bulk_insert_sql)
        result = cursor.executemany(None, tuple_data)
    except:
        print(result)
        cursor.close()
        conn.close()
    
    conn.commit()
    cursor.close()
    conn.close()

In [18]:
# dataframe 데이터 -> DB 입력

# bulk_insert(testD_tp)

## 일괄 처리 작업

In [20]:
# 일괄 처리 작업0

# 테이블 공간 용량 늘리기, SQL DEVELPOER 로 작업하기

# select file_name, file_id, tablespace_name, status
# from dba_data_files
# where tablespace_name='SYSTEM';

# alter database datafile 'C:\ORACLEXE\APP\ORACLE\ORADATA\XE\SYSTEM.DBF' resize 1024M;

In [21]:
# 일괄 처리 작업 1

import os
import numpy as np
import pandas as pd
import cx_Oracle

In [22]:
# 일괄 처리 작업 2

def create_table_seq():
    # APT_DEAL_PRICE TABLE
    create_table_sql = """
        CREATE TABLE APT_DEAL_PRICE
        (
            APT_ID NUMBER PRIMARY KEY,
            REGION VARCHAR2(20 CHAR) NOT NULL,
            REGION_SUB VARCHAR2(40 CHAR) DEFAULT ('없음') NOT NULL,
            APT_NAME VARCHAR2(40 CHAR) NOT NULL,
            APT_SIZE NUMBER NOT NULL,
            APT_FLOOR NUMBER NOT NULL,
            CONTRACT_DATE DATE NOT NULL,
            CONTRACT_PRICE NUMBER NOT NULL,
            COMPLETION_YEAR NUMBER NOT NULL,
            STREET_NUM VARCHAR2(20 CHAR) DEFAULT('없음') NOT NULL,
            STREET_NAME VARCHAR2(40 CHAR) DEFAULT ('없음') NOT NULL
        )
        """

    # FOR APT_ID, AUTO INCREMENT ID
    create_seq_sql = "CREATE SEQUENCE SEQ_ID NOCACHE"

    conn = cx_Oracle.connect('hoseo', 'hoseo', 'localhost:1521/xe')
    cursor = conn.cursor()

    cursor.execute(create_table_sql)
    cursor.execute(create_seq_sql)

    conn.commit()
    cursor.close()
    conn.close()

In [23]:
# 일관 처리 작업 3

def bulk_insert(tuple_data):
    bulk_insert_sql = """
        INSERT INTO APT_DEAL_PRICE
        VALUES (
            SEQ_ID.NEXTVAL, :REGION, :REGION_SUB, :APT_NAME,
            :APT_SIZE, :APT_FLOOR, :CONTRACT_DATE, :CONTRACT_PRICE,
            :COMPLETION_YEAR, :STREET_NUM, :STREET_NAME)
        """

    conn = cx_Oracle.connect('hoseo', 'hoseo', 'localhost:1521/xe')
    cursor = conn.cursor()

    try:
        cursor.prepare(bulk_insert_sql)
        result = cursor.executemany(None, tuple_data)
    except:
        print(result)
        cursor.close()
        conn.close()
    
    conn.commit()
    cursor.close()
    conn.close()

In [24]:
# 일관 처리 작업 4

def batch_csv_to_db():
    
    DATA_FILES_PATH = 'data-files'

    data_files = os.listdir(DATA_FILES_PATH)
    data_files = [ f'{DATA_FILES_PATH}/{r}' for r in data_files ]
   
    create_table_seq()

    total_count = 0

    for df_path in data_files:
        dataD = pd.read_csv(df_path, encoding='utf-8')
        dataD['street_num'].fillna('없음', inplace=True)
        dataD['street_name'].fillna('없음', inplace=True)        
        dataD_tp = [ tuple(r) for r in dataD.values ]

        bulk_insert(dataD_tp)

        print(f'insert complete {df_path}')
        total_count += 1

    print(f'batch complete!! csv count{total_count}')

In [5]:
# 일관 처리 작업 5

# batch_csv_to_db()

insert complete data-files/201208.csv
insert complete data-files/201209.csv
insert complete data-files/201210.csv
insert complete data-files/201211.csv
insert complete data-files/201212.csv
insert complete data-files/201301.csv
insert complete data-files/201302.csv
insert complete data-files/201303.csv
insert complete data-files/201304.csv
insert complete data-files/201305.csv
insert complete data-files/201306.csv
insert complete data-files/201307.csv
insert complete data-files/201308.csv
insert complete data-files/201309.csv
insert complete data-files/201310.csv
insert complete data-files/201311.csv
insert complete data-files/201312.csv
insert complete data-files/201401.csv
insert complete data-files/201402.csv
insert complete data-files/201403.csv
insert complete data-files/201404.csv
insert complete data-files/201405.csv
insert complete data-files/201406.csv
insert complete data-files/201407.csv
insert complete data-files/201408.csv
insert complete data-files/201409.csv
insert compl